# GPT2 Chinese Idiom

## Import

In [1]:
!pip install -Uqq fastai
!pip install transformers

     |████████████████████████████████| 6.8 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 171.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 100.5 MB/s eta 0:00:01


In [3]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

import torch
from fastai.text.all import *

import pandas as pd

### Model1: Nazi Generator

In [11]:
# https://www.kaggle.com/code/vyombhatia/nazi-generator

pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [12]:
# Encoding
ids = tokenizer.encode('This is an example of text, and')
ids

[1212, 318, 281, 1672, 286, 2420, 11, 290]

In [13]:
# Decoding
tokenizer.decode(ids)

'This is an example of text, and'

### Model2: uer/gpt2-chinese-cluecorpussmall

In [83]:
# https://huggingface.co/uer/gpt2-chinese-cluecorpussmall?text=%E4%BB%8A%E5%A4%A9%E5%A4%A9%E6%B0%94%E7%9C%9F%E5%A5%BD%EF%BC%8C%E4%BD%86%E6%98%AF

from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline

tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-chinese-cluecorpussmall")
text_generator = TextGenerationPipeline(model, tokenizer)   
text_generator("这是很久之前的事情了", max_length=100, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '这是很久之前的事情了 ， 一 直 没 在 意 过 。 小 刘 说 ， 以 前 上 厕 所 很 麻 烦 ， 就 是 觉 得 有 必 要 找 个 厕 所 之 类 的 ， 后 来 才 敢 去 找 ， 就 没 再 用 过 。 王 先 生 一 直 不 相 信 ， 为 什 么 要 找 这 样 的 厕 所 ， 他 找 来 小 编 ， 回 答 ： 没 有 的 ， 只 有 别 人 的 厕'}]

### Model3: yuanzhoulvpi/gpt2_chinese

In [4]:
# https://huggingface.co/yuanzhoulvpi/gpt2_chinese?text=%E6%88%91%E5%8F%AB%E7%8E%9B%E4%B8%BD%E4%BA%9A%EF%BC%8C%E6%88%91%E6%9C%80%E5%96%9C%E6%AC%A2%E7%9A%84+%E6%BC%94+%E5%91%98+%E6%98%AF+%E8%B0%81+%E5%95%8A+%EF%BC%9F%2C+%E7%AE%80+%E5%8D%95+%E7%9A%84+%E6%8F%8F+%E8%BF%B0+%E6%98%AF%2C+%E7%AD%94+%E6%A1%88+%E6%98%AF+%E4%BD%A0+%E5%A5%BD+%EF%BC%8C+%E6%88%91+%E6%9D%A5+%E6%8A%8A+%E4%BD%A0+%E4%BB%8B+%E7%BB%8D+%E4%B8%80+%E4%B8%8B..+%E6%98%AF+%E5%A5%B9+%E7%AC%AC+%E4%B8%80+%E4%BD%8D+%E6%BC%94

from transformers import GPT2LMHeadModel, AutoTokenizer

model_name_or_path = "yuanzhoulvpi/gpt2_chinese"#"checkpoint-36000"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, pad_token_id=tokenizer.eos_token_id)

### Model4: IDEA-CCNL/Wenzhong2.0-GPT2-3.5B-chinese

In [68]:
# https://huggingface.co/IDEA-CCNL/Wenzhong2.0-GPT2-3.5B-chinese?text=%E6%88%91%E4%BB%8A%E5%A4%A9%E5%8E%BB%E5%90%83%E8%82%AF%E5%BE%B7%E5%9F%BA%EF%BC%8C%E7%84%B6%E5%90%8E

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# tokenizer = GPT2Tokenizer.from_pretrained('IDEA-CCNL/Wenzhong2.0-GPT2-3.5B-chinese')
# model = GPT2LMHeadModel.from_pretrained('IDEA-CCNL/Wenzhong2.0-GPT2-3.5B-chinese')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

## Preprocessing

### Load data in dictionary format

In [78]:
def read_data(file):
    with open (file) as t:
        data = t.readlines()
    return data

train_data = read_data('./ChID_data/train_data.txt')
dev_data = read_data('./ChID_data/dev_data.txt')
test_data = read_data('./ChID_data/test_data.txt')

train_data[:5]


['{"groundTruth": ["发扬光大", "平易近人", "温文尔雅"], "candidates": [["意气风发", "街谈巷议", "人才辈出", "一脉相传", "后继有人", "发扬光大", "腥风血雨"], ["平易近人", "落落大方", "八仙过海", "彬彬有礼", "史无前例", "盛气凌人", "好自为之"], ["不拘小节", "风流潇洒", "无病呻吟", "言谈举止", "壮志凌云", "关门闭户", "温文尔雅"]], "content": "由实力派演员刘威饰演的清华第三任校长蒋南翔，是我国著名的青年运动家和教育家，他跟清华终身校长梅贻琦一样，都是由清华人自己培养出来的校长。历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的#idiom#，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他#idiom#的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，#idiom#的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。", "realCount": 3}\n',
 '{"groundTruth": ["肥头大耳"], "candidates": [["超凡入圣", "骨瘦如柴", "青面獠牙", "虎背熊腰", "成人之美", "肥头大耳", "神不守舍"]], "content": "#idiom#的掌柜只穿一件衬衫，坐在柜台里。几个堂倌穿着脏得发黑的白工作服，因为没有顾客，都散坐在桌子旁。这当儿看到这位不寻常的客人，都露出好奇的神色列宁曾批评他理论上的错误，同时认为他“所写的全部哲学，赶紧迎上前来伺候。聂赫留朵夫要了一瓶矿泉水，在离窗较远的地方挨着一

In [79]:
# replace #idiom# with candidates
def preprocess(data):
    text_input = []
    idiom_output = []
    for i in range(len(data)):
        data[i] = eval(data[i])
        input_text = data[i]['content']
        ground_truth = data[i]['groundTruth']
        candidates = data[i]['candidates']
        
        for j, idiom in enumerate(ground_truth):
            candidates_str = '|'.join([c for c in candidates[j]])
            input_text = input_text.replace('#idiom#', "(" + candidates_str + ")", 1)
            label = idiom
            
        text_input.append(input_text)
        idiom_output.append(ground_truth)
    return text_input, idiom_output

train_input, train_output = preprocess(train_data)
dev_input, dev_output = preprocess(dev_data)
test_input, test_output = preprocess(test_data)

print(train_input[:2], '\n', train_output[:2])

['由实力派演员刘威饰演的清华第三任校长蒋南翔，是我国著名的青年运动家和教育家，他跟清华终身校长梅贻琦一样，都是由清华人自己培养出来的校长。历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他(平易近人|落落大方|八仙过海|彬彬有礼|史无前例|盛气凌人|好自为之)的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，(不拘小节|风流潇洒|无病呻吟|言谈举止|壮志凌云|关门闭户|温文尔雅)的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。', '(超凡入圣|骨瘦如柴|青面獠牙|虎背熊腰|成人之美|肥头大耳|神不守舍)的掌柜只穿一件衬衫，坐在柜台里。几个堂倌穿着脏得发黑的白工作服，因为没有顾客，都散坐在桌子旁。这当儿看到这位不寻常的客人，都露出好奇的神色列宁曾批评他理论上的错误，同时认为他“所写的全部哲学，赶紧迎上前来伺候。聂赫留朵夫要了一瓶矿泉水，在离窗较远的地方挨着一张铺有肮脏桌布的小桌坐下。'] 
 [['发扬光大', '平易近人', '温文尔雅'], ['肥头大耳']]


In [84]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def postprocess(text):
    return text.replace('.', '').replace('</>','')

def answer_fn(text, top_k=50):
    encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=256, return_tensors="pt").to(device) 
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_length=512,temperature=0.5,do_sample=True,repetition_penalty=3.0 ,top_k=top_k)
    result = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
    return postprocess(result[0])

x1 = """世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、（凭空捏造|高头大马|通力合作|同舟共济|和衷共济|蓬头垢面|紧锣密鼓）。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、(叫苦连天|量体裁衣|金榜题名|百战不殆|知彼知己|风流才子)、
灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。请从括号里选择出合适的成语"""
# y1 = ["高头大马", "叫苦连天"]

result=answer_fn(x1, top_k=50)
print("model result:",result)
print('*'*100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


model result: 世 锦 赛 的 整 体 水 平 远 高 于 亚 洲 杯 ， 要 如 同 亚 洲 杯 那 样 鱼 与 熊 掌 兼 得 ， 就 需 要 各 方 面 密 切 配 合 、 （ 凭 空 捏 造 | 高 头 大 马 | 通 力 合 作 | 同 舟 共 济 | 和 衷 共 济 | 蓬 头 垢 面 | 紧 锣 密 鼓 ） 。 作 为 主 帅 的 俞 觉 敏 ， 除 了 得 打 破 保 守 思 想 ， 敢 于 破 格 用 人 ， 还 得 巧 于 用 兵 、 ( 叫 苦 连 天 | 量 体 裁 衣 | 金 榜 题 名 | 百 战 不 殆 | 知 彼 知 己 | 风 流 才 子 ) 、 灵 活 排 阵 ， 指 挥 得 当 ， 力 争 通 过 比 赛 推 新 人 、 出 佳 绩 、 出 新 的 战 斗 力 。 请 从 括 号 里 选 择 出 合 适 的 成 语? 【 中 国 乒 乓 球 队 】 ： 「 你 们 一 定 是 来 自 日 本 ！ 」 のともにかくるスタートンエイリアーフースタインクッカーをていろのコミーンムールートライトワーク 地 址 : www east collenja - parking station102 @ distorenation203 ＠↗121d270a9b8e6c4afa201401718acuisimple mapsto _ 6think16392093888538358527855695759674684568168805a5007077947367a10150885547786448248497072873789756673654596647497629334199128763340950694321499230541983926295061420161122022551240143004700463602103012710111233601331009013010110173651512016◆37015016010010130500800900340905208887503d2010200025020020152011201220092013 年 11 月 22 到 12 期 间 我 在 这 个 时 候 看 见 有 些 家 伙 把 小 孩 弄 丢 掉 呢 ？ 或 者 干 脆 将 他 扔 进 垃 圾 桶 裡 吧 啦 喏 

### Load data in dataframe format

In [113]:
import pandas as pd
import json

def read_df(file):  
    data_list = []
    with open(file, 'r') as file:
        for line in file:
            data = json.loads(line)
            data_list.append(data)

    df = pd.DataFrame(data_list, columns=['groundTruth', 'candidates', 'content'])
    return df

df_train = read_df('./ChID_data/train_data.txt')
df_dev = read_df('./ChID_data/dev_data.txt')
df_test = read_df('./ChID_data/test_data.txt')

df_train

,groundTruth,candidates,content
0,"[发扬光大, 平易近人, 温文尔雅]","[[意气风发, 街谈巷议, 人才辈出, 一脉相传, 后继有人, 发扬光大, 腥风血雨], [平易近人, 落落大方, 八仙过海, 彬彬有礼, 史无前例, 盛气凌人, 好自为之], [不拘小节, 风流潇洒, 无病呻吟, 言谈举止, 壮志凌云, 关门闭户, 温文尔雅]]",由实力派演员刘威饰演的清华第三任校长蒋南翔，是我国著名的青年运动家和教育家，他跟清华终身校长梅贻琦一样，都是由清华人自己培养出来的校长。历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的#idiom#，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他#idiom#的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，#idiom#的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。
1,[肥头大耳],"[[超凡入圣, 骨瘦如柴, 青面獠牙, 虎背熊腰, 成人之美, 肥头大耳, 神不守舍]]",#idiom#的掌柜只穿一件衬衫，坐在柜台里。几个堂倌穿着脏得发黑的白工作服，因为没有顾客，都散坐在桌子旁。这当儿看到这位不寻常的客人，都露出好奇的神色列宁曾批评他理论上的错误，同时认为他“所写的全部哲学，赶紧迎上前来伺候。聂赫留朵夫要了一瓶矿泉水，在离窗较远的地方挨着一张铺有肮脏桌布的小桌坐下。
2,"[通力合作, 有的放矢]","[[凭空捏造, 高头大马, 通力合作, 同舟共济, 和衷共济, 蓬头垢面, 紧锣密鼓], [叫苦连天, 量体裁衣, 金榜题名, 百战不殆, 知彼知己, 有的放矢, 风流才子]]",世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。
3,[粉墨登场],"[[望洋而叹, 层出不穷, 纷至沓来, 不闻不问, 风云际会, 心广体胖, 粉墨登场]]",2011年，李晨的两部力作《奋斗》、《建党伟业》将相继上映，电视剧《风车》和《理发师》也将#idiom#，承接2010年的辉煌绽放，在新的一年，李晨将展现出更精彩，更惊艳的熟男魅力。
4,[老生常谈],"[[陈词滥调, 赏罚分明, 说来说去, 古往今来, 老生常谈, 双喜临门, 颠扑不破]]",第四，其实以上都是#idiom#，最重要的是你要爱上他，但当你的确有了一段以上的冷静后，我想你应该爆发一下，例如：是用恶搞恶唱的形式，还是在一个空间里将其痛骂，直至心中的闷火发泄出来，以上2点是必不可少的，作用甚佳。
...,...,...,...
520706,[训练有素],"[[不同凡响, 训练有素, 人多势众, 正颜厉色, 溃不成军, 以一当十, 暴风骤雨]]",波托斯胆怯地啃着他的鸡翅膀，当他感到桌底下诉讼代理人太太的膝盖总来触碰他的膝盖时，他为之一颤。他也将倍受珍惜的这种葡萄酒喝了半杯，他品出这是蒙特勒伊产的难喝的酒。#idiom#的味觉真厉害。科克纳尔先生瞅他猛灌这种纯葡萄酒，长叹一声。
520707,"[不胫而走, 添油加醋, 道听涂说, 应有尽有, 穷愁潦倒]","[[有勇无谋, 不胫而走, 摧枯拉朽, 轩然大波, 销声匿迹, 重见天日, 不攻自破], [捕风捉影, 料事如神, 原原本本, 怒气冲天, 借题发挥, 添油加醋, 衣衫蓝缕], [司空见惯, 不足为凭, 无稽之言, 道听涂说, 人云亦云, 劳苦功高, 高头大马], [五花八门, 林林总总, 开门见山, 一命归阴, 洗心革面, 数不胜数, 应有尽有], [家徒四壁, 穷愁潦倒, 欢欣鼓舞, 安然无恙, 一贫如洗, 囊空如洗, 粉墨登场]]",帕特里克被抓获的消息在沿海地区司法界#idiom#。许许多多的律师都在谈论此事，而且往往要#idiom#；然后这些经过加工的事实又以惊人的速度在他们中间再次流传。各种各样的传闻，#idiom#的，完全杜撰的，#idiom#。他体重130磅，会说五种语言。那笔巨款已经找到。所有那么多钱都收不回来了。他几乎是#idiom#。或许他有一幢豪华住宅。他一人独住。他又娶了妻，养了三个孩子。有关方面已经查明了那笔巨款的下落。迄今他们没有任何线索。
520708,[狭路相逢],"[[决一雌雄, 流连忘反, 冤家路窄, 狭路相逢, 顺理成章, 一文不名, 一决雌雄]]",新浪体育讯9月27日消息：2011年中信银行中国业余高尔夫球巡回赛首轮比赛正在苏州新东海高尔夫乡村俱乐部举行，女子球员上午出发，目前已全部完成比赛，刘思芸交出女选手中唯一六字头69杆，2杆优势获单独领先。业巡赛沈阳站领先争冠好手吴莎(微博)杨嘉欣(微博)再度#idiom#，两人都交出71杆后，排在并列第二位。
520709,"[影影绰绰, 感同身受]","[[楚楚动人, 亭台楼阁, 无边无际, 戛然而止, 星星点点, 影影绰绰, 大势所趋], [刻骨铭心, 将心比心, 藏龙卧虎, 明火执仗, 盖棺论定, 感同身受, 肝肠寸断]]",雨肆虐着，天像被捅破了一般。又是一个清凉的夏夜，留出很多空白来，等着人们去填写和作答。睡不踏实，左小青早早上了床，挪过电视机，将荧屏正对着，一遍遍地去读各个频道轮番播出的抢劫画面。画面资料被重新制作了一遍，显得很新潮，但肖依被枪杀的片段仍打上了马赛克，#idiom#的。新凯悦珠宝店一案的悬红30万，被讲究地制成了艺术字，黑底红字，绷紧了整个屏幕，还尾随着三个大大的惊叹号。左小青很起腻，她觉得电视台根本不负责任，将一场抢劫谋杀案搞成了狂欢节，唯恐天下不乱似的。她#idiom#地想，要是被害人家属看到如此的画面，该作何感想?


In [114]:
# change the candidates format. Call this cell only once.
def format_candidates(candidates):
    return '(' + '|'.join(candidates[0]) + ')'

df_train['candidates'] = df_train['candidates'].apply(format_candidates)

# replace #idiom# in content with the candidates
df_train['content'] = df_train.apply(lambda row: row['content'].replace('#idiom#', row['candidates']), axis=1)

# delete candidates column
df_train = df_train.loc[:, ['groundTruth', 'content']]

In [115]:
df_train

,groundTruth,content
0,"[发扬光大, 平易近人, 温文尔雅]",由实力派演员刘威饰演的清华第三任校长蒋南翔，是我国著名的青年运动家和教育家，他跟清华终身校长梅贻琦一样，都是由清华人自己培养出来的校长。历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。
1,[肥头大耳],(超凡入圣|骨瘦如柴|青面獠牙|虎背熊腰|成人之美|肥头大耳|神不守舍)的掌柜只穿一件衬衫，坐在柜台里。几个堂倌穿着脏得发黑的白工作服，因为没有顾客，都散坐在桌子旁。这当儿看到这位不寻常的客人，都露出好奇的神色列宁曾批评他理论上的错误，同时认为他“所写的全部哲学，赶紧迎上前来伺候。聂赫留朵夫要了一瓶矿泉水，在离窗较远的地方挨着一张铺有肮脏桌布的小桌坐下。
2,"[通力合作, 有的放矢]",世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、(凭空捏造|高头大马|通力合作|同舟共济|和衷共济|蓬头垢面|紧锣密鼓)。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、(凭空捏造|高头大马|通力合作|同舟共济|和衷共济|蓬头垢面|紧锣密鼓)、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。
3,[粉墨登场],2011年，李晨的两部力作《奋斗》、《建党伟业》将相继上映，电视剧《风车》和《理发师》也将(望洋而叹|层出不穷|纷至沓来|不闻不问|风云际会|心广体胖|粉墨登场)，承接2010年的辉煌绽放，在新的一年，李晨将展现出更精彩，更惊艳的熟男魅力。
4,[老生常谈],第四，其实以上都是(陈词滥调|赏罚分明|说来说去|古往今来|老生常谈|双喜临门|颠扑不破)，最重要的是你要爱上他，但当你的确有了一段以上的冷静后，我想你应该爆发一下，例如：是用恶搞恶唱的形式，还是在一个空间里将其痛骂，直至心中的闷火发泄出来，以上2点是必不可少的，作用甚佳。
...,...,...
520706,[训练有素],波托斯胆怯地啃着他的鸡翅膀，当他感到桌底下诉讼代理人太太的膝盖总来触碰他的膝盖时，他为之一颤。他也将倍受珍惜的这种葡萄酒喝了半杯，他品出这是蒙特勒伊产的难喝的酒。(不同凡响|训练有素|人多势众|正颜厉色|溃不成军|以一当十|暴风骤雨)的味觉真厉害。科克纳尔先生瞅他猛灌这种纯葡萄酒，长叹一声。
520707,"[不胫而走, 添油加醋, 道听涂说, 应有尽有, 穷愁潦倒]",帕特里克被抓获的消息在沿海地区司法界(有勇无谋|不胫而走|摧枯拉朽|轩然大波|销声匿迹|重见天日|不攻自破)。许许多多的律师都在谈论此事，而且往往要(有勇无谋|不胫而走|摧枯拉朽|轩然大波|销声匿迹|重见天日|不攻自破)；然后这些经过加工的事实又以惊人的速度在他们中间再次流传。各种各样的传闻，(有勇无谋|不胫而走|摧枯拉朽|轩然大波|销声匿迹|重见天日|不攻自破)的，完全杜撰的，(有勇无谋|不胫而走|摧枯拉朽|轩然大波|销声匿迹|重见天日|不攻自破)。他体重130磅，会说五种语言。那笔巨款已经找到。所有那么多钱都收不回来了。他几乎是(有勇无谋|不胫而走|摧枯拉朽|轩然大波|销声匿迹|重见天日|不攻自破)。或许他有一幢豪华住宅。他一人独住。他又娶了妻，养了三个孩子。有关方面已经查明了那笔巨款的下落。迄今他们没有任何线索。
520708,[狭路相逢],新浪体育讯9月27日消息：2011年中信银行中国业余高尔夫球巡回赛首轮比赛正在苏州新东海高尔夫乡村俱乐部举行，女子球员上午出发，目前已全部完成比赛，刘思芸交出女选手中唯一六字头69杆，2杆优势获单独领先。业巡赛沈阳站领先争冠好手吴莎(微博)杨嘉欣(微博)再度(决一雌雄|流连忘反|冤家路窄|狭路相逢|顺理成章|一文不名|一决雌雄)，两人都交出71杆后，排在并列第二位。
520709,"[影影绰绰, 感同身受]",雨肆虐着，天像被捅破了一般。又是一个清凉的夏夜，留出很多空白来，等着人们去填写和作答。睡不踏实，左小青早早上了床，挪过电视机，将荧屏正对着，一遍遍地去读各个频道轮番播出的抢劫画面。画面资料被重新制作了一遍，显得很新潮，但肖依被枪杀的片段仍打上了马赛克，(楚楚动人|亭台楼阁|无边无际|戛然而止|星星点点|影影绰绰|大势所趋)的。新凯悦珠宝店一案的悬红30万，被讲究地制成了艺术字，黑底红字，绷紧了整个屏幕，还尾随着三个大大的惊叹号。左小青很起腻，她觉得电视台根本不负责任，将一场抢劫谋杀案搞成了狂欢节，唯恐天下不乱似的。她(楚楚动人|亭台楼阁|无边无际|戛然而止|星星点点|影影绰绰|大势所趋)地想，要是被害人家属看到如此的画面，该作何感想?


In [125]:
all_texts = np.array(df_train['content'])

In [126]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [127]:
# Defining the splits for the dataloader:
splits = [range_of(df_train), list(range(len(df_train), len(all_texts)))]
# print(splits)

# Defining the Transformed Lists:
tls = TfmdLists(all_texts, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

In [129]:
show_at(tls.train, 0)

由 实 力 派 演 员 刘 威 饰 演 的 清 华 第 三 任 校 长 蒋 南 翔 ， 是 我 国 著 名 的 青 年 运 动 家 和 教 育 家 ， 他 跟 清 华 终 身 校 长 梅 贻 琦 一 样 ， 都 是 由 清 华 人 自 己 培 养 出 来 的 校 长 。 历 史 上 的 蒋 南 翔 是 著 名 的 [UNK] 一 二 九 [UNK] 学 生 救 亡 运 动 的 领 导 人 之 一 ， 他 在 清 华 校 长 之 位 14 年 期 间 ， 不 但 很 好 的 继 承 了 清 华 建 校 之 初 的 优 秀 传 统 与 理 念 ， 而 且 更 加 的 ( 意 气 风 发 | 街 谈 巷 议 | 人 才 辈 出 | 一 脉 相 传 | 后 继 有 人 | 发 扬 光 大 | 腥 风 血 雨 ) ， 他 把 清 华 的 教 师 队 伍 扩 大 了 将 近 5 倍 ， 将 清 华 本 科 人 数 破 万 ， 为 新 中 国 培 养 了 大 量 的 有 用 人 才 。 在 《 天 行 健 》 中 饰 演 蒋 南 翔 的 刘 威 是 观 众 所 熟 悉 的 著 名 实 力 派 演 员 ， 早 在 1987 年 刘 威 就 在 《 关 东 大 侠 》 中 饰 演 豪 爽 仗 义 的 关 云 天 一 角 而 获 得 了 金 鸡 奖 最 佳 男 主 角 的 提 名 ， 后 来 更 是 因 在 《 唐 明 皇 》 中 精 湛 的 表 演 而 一 举 夺 得 金 鹰 奖 最 佳 男 演 员 奖 。 此 次 《 天 行 健 》 选 定 刘 威 来 出 演 正 是 看 中 了 他 ( 意 气 风 发 | 街 谈 巷 议 | 人 才 辈 出 | 一 脉 相 传 | 后 继 有 人 | 发 扬 光 大 | 腥 风 血 雨 ) 的 表 演 方 式 和 对 人 物 深 入 内 心 的 刻 画 。 至 此 ， 《 天 行 健 》 中 涉 及 的 三 位 清 华 校 长 的 人 选 都 已 经 曝 光 ， ( 意 气 风 发 | 街 谈 巷 议 | 人 才 辈 出 | 一 脉 相 传 | 后 继 有 人 | 发 扬 光 大 | 腥 风 血 雨 ) 的 第 一 任 校 长 赵 文? 、 稳 重 坚 毅 的 第 二 任 校 长 孙 逊 、 亲 切 务 实 的 第 三 任 校 长 刘 威 ， 再

In [130]:
# bs refers to the Batch Size while sl refers to the sequence length:
bs,sl = 4,240 

# Defining the Dataloader:
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [131]:
dls.show_batch(max_n=5)

,text,text_
0,2006 年 12 月 ， 就 在 张 坤 龙 为 自 己 近 百 万 元 欠 款 ( 不 胜 其 烦 | 高 歌 猛 进 | 人 心 惶 惶 | 一 筹 莫 展 | 落 落 大 方 | 焦 头 烂 额 | 好 好 先 生 ) 之 际 ， 一 个 转 机 又 出 现 了 ： 政 府 要 拆 迁 他 所 经 营 的 工 厂 ， 其 2100 平 方 米 的 厂 房 面 积 获 得 了 550 万 元 的 拆 迁 补 偿 款 。 还 掉 欠 债 后 ， 张 坤 龙 本 想 做 些 正 经 生 意 ， 却 遭 遇 到 金 融 危 机 ， 几 次 投 资 都 蒙 受 败 绩 。 子 不 久 就 软 下 来 了 。 她,年 12 月 ， 就 在 张 坤 龙 为 自 己 近 百 万 元 欠 款 ( 不 胜 其 烦 | 高 歌 猛 进 | 人 心 惶 惶 | 一 筹 莫 展 | 落 落 大 方 | 焦 头 烂 额 | 好 好 先 生 ) 之 际 ， 一 个 转 机 又 出 现 了 ： 政 府 要 拆 迁 他 所 经 营 的 工 厂 ， 其 2100 平 方 米 的 厂 房 面 积 获 得 了 550 万 元 的 拆 迁 补 偿 款 。 还 掉 欠 债 后 ， 张 坤 龙 本 想 做 些 正 经 生 意 ， 却 遭 遇 到 金 融 危 机 ， 几 次 投 资 都 蒙 受 败 绩 。 子 不 久 就 软 下 来 了 。 她 (
1,为 有 必 要 在 这 里 介 绍 一 下 它 的 要 点 。 塔 鲁 对 这 位 靠 年 金 生 活 的 小 矮 个 儿 的 总 印 象 可 以 归 纳 为 一 句 话 ： [UNK] 这 是 一 个 形 象 正 在 高 大 起 来 的 人 物 。 [UNK] 至 少 从 表 面 上 看 来 ， 他 的 心 情 显 得 越 来 越 愉 快 。 他 对 事 态 的 发 展 并 无 不 满 ， 在 塔 鲁 面 前 ， 他 有 时 会 用 这 类 话 来 表 达 他 思 想 深 处 的 想 法 ： [UNK] 当 然 ， 情 况 仍 不 见 好 转 ， 不 过 至 少 大 家 是 ( 肝 胆 相 照 | 上 下 一 心,有 必 要 在 这 里 介 绍 一 下 它 的 要 点 。 塔 鲁 对 这 位 靠 年 金 生 活 的 小 矮 个 儿 的 总 印 象 可 以 归 纳 为 一 句 话 ： [UNK] 这 是 一 个 形 象 正 在 高 大 起 来 的 人 物 。 [UNK] 至 少 从 表 面 上 看 来 ， 他 的 心 情 显 得 越 来 越 愉 快 。 他 对 事 态 的 发 展 并 无 不 满 ， 在 塔 鲁 面 前 ， 他 有 时 会 用 这 类 话 来 表 达 他 思 想 深 处 的 想 法 ： [UNK] 当 然 ， 情 况 仍 不 见 好 转 ， 不 过 至 少 大 家 是 ( 肝 胆 相 照 | 上 下 一 心 |
2,面 做 出 一 些 贡 献 ！ [UNK] ( 矫 枉 过 正 | 颠 扑 不 破 | 今 生 今 世 | 操 之 过 急 | 格 杀 勿 论 | 突 飞 猛 进 | 物 极 必 反 ) 。 昨 日 [UNK] 除 1 年 期 品 种 外 全 线 回 落 。 其 中 ， 隔 夜 利 率 继 上 周 五 狂 泻 201. 71 个 基 点 后 ， 昨 天 再 跌 28. 62 个 基 点 ， 至 2. 55 % ； 而 7 日 利 率 大 跌 119. 5 个 基 点 ， 至 3. 2008 % 的 两 周 新 低 。 [UNK] 的 跳 水 ， 反 映 市 场 流 动 性 回 暖 。 作 为 该 次 活 动 的 颁 奖 嘉 宾 童 蕾 向 我 们,做 出 一 些 贡 献 ！ [UNK] ( 矫 枉 过 正 | 颠 扑 不 破 | 今 生 今 世 | 操 之 过 急 | 格 杀 勿 论 | 突 飞 猛 进 | 物 极 必 反 ) 。 昨 日 [UNK] 除 1 年 期 品 种 外 全 线 回 落 。 其 中 ， 隔 夜 利 率 继 上 周 五 狂 泻 201. 71 个 基 点 后 ， 昨 天 再 跌 28. 62 个 基 点 ， 至 2. 55 % ； 而 7 日 利 率 大 跌 119. 5 个 基 点 ， 至 3. 2008 % 的 两 周 新 低 。 [UNK] 的 跳 水 ， 反 映 市 场 流 动 性 回 暖 。 作 为 该 次 活 动 的 颁 奖 嘉 宾 童 蕾 向 我 们 道
3,冷 漠 严 肃 的 样 子 说 。 2 月 10 日 晚 10 时 许 ， 一 名 手 提 猎 枪 的 男 子 出 现 在 坑 梓 办 事 处 第 三 办 公 楼 ， 在 寻 了 一 圈 无 果 之 后 ， 提 着 枪 ( 寸 步 难 行 | 一 丝 不 苟 | 不 可 磨 灭 | 趾 高 气 扬 | 光 明 正 大 | 鬼 鬼 祟 祟 | 大 摇 大 摆 ) 地 走 出 大 门 。 原 来 ， 男 子 是 来 [UNK] 寻 仇 [UNK] 的 ， 他 深 恨 街 道 执 法 队 长 总 是 针 对 自 己 叔 叔 家 的 违 建 ， 因 此 带 枪 过 来 寻 找 执 法 队 长 报 仇 ， 并 扬 言 [UNK],漠 严 肃 的 样 子 说 。 2 月 10 日 晚 10 时 许 ， 一 名 手 提 猎 枪 的 男 子 出 现 在 坑 梓 办 事 处 第 三 办 公 楼 ， 在 寻 了 一 圈 无 果 之 后 ， 提 着 枪 ( 寸 步 难 行 | 一 丝 不 苟 | 不 可 磨 灭 | 趾 高 气 扬 | 光 明 正 大 | 鬼 鬼 祟 祟 | 大 摇 大 摆 ) 地 走 出 大 门 。 原 来 ， 男 子 是 来 [UNK] 寻 仇 [UNK] 的 ， 他 深 恨 街 道 执 法 队 长 总 是 针 对 自 己 叔 叔 家 的 违 建 ， 因 此 带 枪 过 来 寻 找 执 法 队 长 报 仇 ， 并 扬 言 [UNK] 要


In [132]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

# Actually Tokenizing everything:
tokenized = [tokenize(t) for t in progress_bar(all_texts)]

## Finetuning

In [133]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [134]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

In [135]:
learn.validate()

/Users/sherrywang/opt/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/sherrywang/opt/anaconda3/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/Users/sherrywang/opt/anaconda3/lib/python3.8/site-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


(#2) [None,None]

In [136]:
# learning rate
learn.lr_find()

KeyboardInterrupt: 

In [ ]:
# fitting the model
learn.fit_one_cycle(5, 1e-5)

epoch,train_loss,valid_loss,perplexity,time
0,9.825723,None,None,00:00
1,10.710649,None,None,00:00
2,10.867957,None,None,00:00
3,10.785975,None,None,00:00
4,10.983597,None,None,00:00


### Prediction

In [ ]:
def predict(prompt, length, beams, temp):
    prompt_ids = tokenizer.encode(prompt)
    inp = tensor(prompt_ids)[None].cuda()
    preds = learn.model.generate(inp, max_length=length, num_beams=beams, temperature=temp)
    return tokenizer.decode(preds[0].cpu().numpy())